# Basic Usage
Here you can learn how to use this package as a python library. You can create simulations, run routing, congestion simulations and other analytics from here.

## Prerequisites

Things you need to define before you can run all the simulation:
- Place name of the format (for example 'Eindhoven, Netherlands')
- The url to the Geofabrik OSM repository for the smallest region covering the place
- The url to the Transitfeed place
- The date of the simulation (specifies the date where the graph will be built. Note, that the corresponding OSM and GTFS data must be available at this time)
- The amount of RAM that can be used to build the graph and run routing (in GB, requirements vary by graph size)
- The amount of threads to use for routing
- The amount of virtual commuters to simulate
- Whether to use delay simulation (you need to setup other data for this)

In [ ]:
import sys
sys.path.append('..')
import hiveline as hl
from datetime import date

Define computation variables

In [ ]:
geofabrik_url = 'https://download.geofabrik.de/europe/netherlands/noord-brabant.html'
transitfeed_url = 'https://transitfeeds.com/l/194-the-netherlands'

simulation_date = date(2022, 10, 4)
num_virtual_commuters = 2000

Define study location and time

In [ ]:
place_name = 'Eindhoven, Netherlands'
year = '2020' # str(simulation_date.year)
place = hl.Place(place_name, year)

In [ ]:
# (optional) add surrounding cities
country = 'Netherlands'
place_names = ['Son en Breugel', 'Nuenen', 'Geldrop-Mierlo', 'Helmond', 'Heeze-Leende', 'Waalre', 'Veldhoven', 'Eersel', 'Oirschot', 'Best']
place_names = [p+', '+country for p in place_names]
place.merge_places('Eindhoven suburbs, Netherlands', place_names)

## Load place ressources

Now we can get the place_id and index the OSM and GTFS data for routing. Note that this may scrape the data from the corresponding websites.

In [ ]:
db = hl.get_database()
place_resources = hl.create_place_resources(geofabrik_url, transitfeed_url, place_name, db=db, skip_existing=True)

## Create and Run Simulation

Now that all data is loaded into the database and built locally, we can create a simulation. This will create a new simulation in the database and create a set of virtual commuters

In [ ]:
sim_id = hl.create_simulation(place, simulation_date, num_virtual_commuters, db=db, use_parking=False)

After creating a simulation, we can run routing on it. This will use a given routing profile to route all virtual commuters and store the results in the database. This may take a while.

In [ ]:
profile = "opentripplanner"
ram_gb = 14
num_threads = 6
use_delay_simulation = False
hl.route_virtual_commuters(sim_id, profile, use_delays=use_delay_simulation, memory_gb=ram_gb, num_threads=num_threads, reset_jobs=True)

## Analyze Simulation

Now that we have all routes, we can analyze them. There are a lot of plotting an analysis methods available. 

In [ ]:
journeys = hl.Journeys(sim_id, db=db, use_cache=False)  # get all journeys to analyze

In [ ]:
stats = hl.get_journeys_stats(journeys)

In [ ]:
stats.to_dict()

In [ ]:
print(stats.get_all_modal_shares())

In [ ]:
print(stats.get_transit_modal_share())  # transit vs car modal share

In [ ]:
hl.plot_monte_carlo_convergence(journeys, city_name=place_name)

In [ ]:
from IPython.display import Image
filename = hl.plot_traces(journeys, place, folder="images", filename=place_name + "-traces")
Image(filename=filename) 

Optionally, you can filter the journeys by a given bounding shape to only account for modal shares in a specific region. This is an example where we use the primary place boundary as a bounding shape. 

In [ ]:
shape = place.shape.iloc[0].geometry
stats = hl.get_journeys_stats(journeys, shape=shape)
print(stats.to_dict())
print(stats.get_all_modal_shares())
print("transit modal share:", stats.get_transit_modal_share())